# Data Engineer & Analyst

Finding just the right data for a project can be a challenge. Finding perfectly clean data ready for machine learning can be impossible. For our first assignment we'll generate some data, but before we do that we'll need a place to store it! For that I've included a MongoDB interface, all you need to do is enter your credentials when prompted.

## MongoDB Database Interface

Our first assignment is to generate some random data, but first we need a place to put it!

MongoDB is a good place to start when learning database operations in Python. Unlike relational databases, working with a no-SQL database (MongoDB) is more like working with other Python libraries and less like writing obscure SQL queries. For some of these assignments you will need a free [MongoDB account](https://www.mongodb.com/). In assignment 2 you'll create your own database interface much like the one below.


In [1]:
# PyMongo requires dnspython to be installed
!pip install dnspython

In [2]:
from typing import Dict, Iterable
from pymongo import MongoClient
import pandas as pd

In [14]:
class MongoDB:

    def __init__(self, url, database, collection):
        self.url = url
        self.database = database
        self.collection = collection

    def connect(self):
        return MongoClient(self.url)[self.database][self.collection]

    def find(self, query_obj: Dict) -> pd.DataFrame:
        return pd.DataFrame(self.connect().find(query_obj))

    def insert(self, insert_obj: Iterable[Dict]):
        self.connect().insert_many(insert_obj)

    def update(self, query: Dict, data: Dict):
        self.connect().update_many(query, {"$set": data})

    def delete(self, query_obj: Dict):
        self.connect().delete_many(query_obj)

    def get_df(self, query_obj: Dict = None) -> pd.DataFrame:
        return pd.DataFrame(self.find(query_obj or {}))

# Mock Data


1. Sign up for MongoDB if you don't already have an account.
2. Run the cells below to get a feel for it. Edit the code, and have fun.
3. Create and store at least 1000 monsters using the database interface and MonsterLab's Monster class.
4. Find all the Dragons, print them as a DataFrame.
5. Get all the monsters into a pandas DataFrame.

Bandersnatch should be very happy, indeed!

It's best practice to store passwords and credentials in a `.env` file. Here the notebook will ask you for your database info when you run the cell below. Make sure you have your MongoDB account setup first.

In [12]:
base_url = input("URL? ")
user_name = input("Username? ")
password = input("Password? ")

database = input("Database? ")  # This can be named anything you like: MonsterLab
collection = input("Collection? ")  # This can be named anything you like: Monsters

url = f"mongodb+srv://{user_name}:{password}@{base_url}"

In [15]:
db = MongoDB(url, database, collection)
print(db.get_df())

Empty DataFrame
Columns: []
Index: []


Now we need some data! Random data can be generated in many ways. Here's an examples using MonsterLab and the database interface above. Mock data should have the same shape as expected in the *real data*. 

[MonsterLab](https://pypi.org/project/MonsterLab/) MonsterLab is how Bandersnatch generates its data, more on that later...

For this assignment review the code cells below and have a go playing with the generators in MonsterLab. See what you can do!

## Random Monsters: MonsterLab

In [16]:
!pip install MonsterLab

In [17]:
from MonsterLab import Monster

In [18]:
help(Monster)

Help on class Monster in module MonsterLab.monster_lab:

class Monster(builtins.object)
 |  Monster(name=None, monster_type=None, level=None, rarity=None)
 |  
 |  Methods defined here:
 |  
 |  __init__(self, name=None, monster_type=None, level=None, rarity=None)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  __repr__(self)
 |      Return repr(self).
 |  
 |  __str__(self)
 |      Return str(self).
 |  
 |  to_dict(self)
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |  
 |  __dict__
 |      dictionary for instance variables (if defined)
 |  
 |  __weakref__
 |      list of weak references to the object (if defined)



### A Random Monster

In [19]:
m1 = Monster()
m1

Name: Wyvern
Type: Dragon
Level: 3
Rarity: Rank 1
Damage: 3d4
Health: 12.85
Energy: 11.02
Sanity: 11.74
Time Stamp: 2022-03-31 07:23:35

### Monster as a Dict

In [20]:
m2 = Monster()
m2.to_dict()

{'Name': 'Succubus',
 'Type': 'Devilkin',
 'Level': 8,
 'Rarity': 'Rank 0',
 'Damage': '8d2+5',
 'Health': 15.78,
 'Energy': 15.92,
 'Sanity': 16.36,
 'Time Stamp': '2022-03-31 07:23:43'}

### Insert a Single Custom Monster into the database

In [21]:
monster = Monster(
    name="Bandersnatch", 
    monster_type="Demonic", 
    level=20, 
    rarity="Rank 5",
)

db.insert([monster.to_dict()])

db.get_df()

,_id,Name,Type,Level,Rarity,Damage,Health,Energy,Sanity,Time Stamp
0,6245b9736c1283d7863dff86,Bandersnatch,Demonic,20,Rank 5,20d12+2,240.43,239.76,240.13,2022-03-31 07:23:47


### Insert Many Random Monsters

In [22]:
n_monsters = 1024
db.insert(Monster().to_dict() for _ in range(n_monsters))

In [23]:
db.get_df()

,_id,Name,Type,Level,Rarity,Damage,Health,Energy,Sanity,Time Stamp
0,6245b9736c1283d7863dff86,Bandersnatch,Demonic,20,Rank 5,20d12+2,240.43,239.76,240.13,2022-03-31 07:23:47
1,6245b97b6c1283d7863dff89,Hell Hound,Demonic,6,Rank 4,6d10,57.97,60.36,56.12,2022-03-31 07:23:55
2,6245b97b6c1283d7863dff8a,Death Knight,Undead,8,Rank 3,8d8+1,63.31,61.66,63.90,2022-03-31 07:23:55
3,6245b97b6c1283d7863dff8b,Dust Spirit,Fey,4,Rank 0,4d2+3,7.73,8.10,8.62,2022-03-31 07:23:55
4,6245b97b6c1283d7863dff8c,Faerie Dragon,Dragon,14,Rank 3,14d8+4,111.46,112.30,110.41,2022-03-31 07:23:55
...,...,...,...,...,...,...,...,...,...,...
1020,6245b97b6c1283d7863e0384,Pit Fiend,Demonic,10,Rank 3,10d8+2,76.99,77.45,78.79,2022-03-31 07:23:55
1021,6245b97b6c1283d7863e0385,Banshee,Undead,2,Rank 1,2d4+1,8.42,9.23,6.26,2022-03-31 07:23:55
1022,6245b97b6c1283d7863e0386,Mud Devil,Devilkin,6,Rank 3,6d8,46.21,51.79,49.64,2022-03-31 07:23:55
1023,6245b97b6c1283d7863e0387,Emerald Demon,Demonic,9,Rank 0,9d2+1,17.02,17.91,18.64,2022-03-31 07:23:55


### Find all the monsters that match a simple query

In [24]:
db.find({"Name": "Vampire"})

,_id,Name,Type,Level,Rarity,Damage,Health,Energy,Sanity,Time Stamp
0,6245b97b6c1283d7863dffc2,Vampire,Undead,10,Rank 1,10d4+3,39.92,38.75,39.90,2022-03-31 07:23:55
1,6245b97b6c1283d7863e00f5,Vampire,Undead,3,Rank 4,3d10+2,32.29,29.52,30.26,2022-03-31 07:23:55
2,6245b97b6c1283d7863e00f9,Vampire,Undead,2,Rank 0,2d2+2,3.31,3.28,4.35,2022-03-31 07:23:55
3,6245b97b6c1283d7863e0153,Vampire,Undead,15,Rank 4,15d10+1,146.77,153.68,146.83,2022-03-31 07:23:55
4,6245b97b6c1283d7863e0156,Vampire,Undead,17,Rank 2,17d6,101.59,103.41,101.02,2022-03-31 07:23:55
5,6245b97b6c1283d7863e018b,Vampire,Undead,11,Rank 1,11d4+5,43.41,43.23,45.60,2022-03-31 07:23:55
6,6245b97b6c1283d7863e0270,Vampire,Undead,3,Rank 3,3d8,22.59,22.37,23.45,2022-03-31 07:23:55
7,6245b97b6c1283d7863e0338,Vampire,Undead,11,Rank 0,11d2,21.94,22.38,21.29,2022-03-31 07:23:55
8,6245b97b6c1283d7863e036d,Vampire,Undead,3,Rank 4,3d10,31.77,30.92,26.38,2022-03-31 07:23:55


### Find the monsters that match a complex query

All Undead with level less than 10 and rarity rank 0.

In [27]:
db.find({"Type": "Undead", "Level": {"$lt": 10}, "Rarity": "Rank 0"})

,_id,Name,Type,Level,Rarity,Damage,Health,Energy,Sanity,Time Stamp
0,6245b97b6c1283d7863dffc9,Lich King,Undead,6,Rank 0,6d2+1,11.48,11.25,11.83,2022-03-31 07:23:55
1,6245b97b6c1283d7863e0017,Revenant,Undead,6,Rank 0,6d2+2,12.46,12.23,12.90,2022-03-31 07:23:55
2,6245b97b6c1283d7863e001d,Revenant,Undead,7,Rank 0,7d2+3,14.21,14.00,14.31,2022-03-31 07:23:55
3,6245b97b6c1283d7863e002c,Demilich,Undead,2,Rank 0,2d2+3,3.20,4.12,3.60,2022-03-31 07:23:55
4,6245b97b6c1283d7863e005f,Revenant,Undead,2,Rank 0,2d2+1,4.41,3.54,4.35,2022-03-31 07:23:55
5,6245b97b6c1283d7863e0068,Zombie Knight,Undead,3,Rank 0,3d2+2,5.30,6.22,5.29,2022-03-31 07:23:55
6,6245b97b6c1283d7863e0096,Skeletal Mage,Undead,2,Rank 0,2d2+1,3.12,4.43,4.94,2022-03-31 07:23:55
7,6245b97b6c1283d7863e0099,Mummy Lord,Undead,9,Rank 0,9d2,18.00,17.83,18.55,2022-03-31 07:23:55
8,6245b97b6c1283d7863e00e2,Lich,Undead,7,Rank 0,7d2+2,13.67,13.49,14.09,2022-03-31 07:23:55
9,6245b97b6c1283d7863e00f9,Vampire,Undead,2,Rank 0,2d2+2,3.31,3.28,4.35,2022-03-31 07:23:55


### Get all the monsters into a pandas dataframe.

In [28]:
vampires = db.get_df({"Name": "Vampire"})
vampires

,_id,Name,Type,Level,Rarity,Damage,Health,Energy,Sanity,Time Stamp
0,6245b97b6c1283d7863dffc2,Vampire,Undead,10,Rank 1,10d4+3,39.92,38.75,39.90,2022-03-31 07:23:55
1,6245b97b6c1283d7863e00f5,Vampire,Undead,3,Rank 4,3d10+2,32.29,29.52,30.26,2022-03-31 07:23:55
2,6245b97b6c1283d7863e00f9,Vampire,Undead,2,Rank 0,2d2+2,3.31,3.28,4.35,2022-03-31 07:23:55
3,6245b97b6c1283d7863e0153,Vampire,Undead,15,Rank 4,15d10+1,146.77,153.68,146.83,2022-03-31 07:23:55
4,6245b97b6c1283d7863e0156,Vampire,Undead,17,Rank 2,17d6,101.59,103.41,101.02,2022-03-31 07:23:55
5,6245b97b6c1283d7863e018b,Vampire,Undead,11,Rank 1,11d4+5,43.41,43.23,45.60,2022-03-31 07:23:55
6,6245b97b6c1283d7863e0270,Vampire,Undead,3,Rank 3,3d8,22.59,22.37,23.45,2022-03-31 07:23:55
7,6245b97b6c1283d7863e0338,Vampire,Undead,11,Rank 0,11d2,21.94,22.38,21.29,2022-03-31 07:23:55
8,6245b97b6c1283d7863e036d,Vampire,Undead,3,Rank 4,3d10,31.77,30.92,26.38,2022-03-31 07:23:55


# Assignment 3: Visualizations

In [29]:
import plotly.graph_objects as go
import plotly.express as px

In [30]:
def rank_lookup(rank: str) -> str:
    return {
        "Rank 0": "Very Common",
        "Rank 1": "Common",
        "Rank 2": "Uncommon",
        "Rank 3": "Rare",
        "Rank 4": "Epic",
        "Rank 5": "Legendary",
    }.get(rank, "Unknown")

## Pie Chart: Monsters by Rarity

In [33]:
target = "Rarity"

df = db.get_df()[target].value_counts()
df.index = [rank_lookup(col) for col in df.index]
data = go.Pie(labels=df.index, values=df.values, hole=0.5)

layout = go.Layout(
    title=f"Monsters by {target}",
    colorway=px.colors.qualitative.Antique,
    height=700,
    width=770,
)

figure = go.Figure(data, layout)

figure.update_traces(
    textfont_size=14,
    textinfo='percent+label',
)

figure.show()

## Line Chart: Monster Rarity Totals Over Time

In [35]:
from itertools import accumulate

In [39]:
target = "Rarity"  # ["Rarity", "Level", "Type"]

df = db.get_df()

df_cross = pd.crosstab(df['Time Stamp'], df[target])

for column in df_cross.columns:
    df_cross[column] = list(accumulate(df_cross[column]))

title = f"Monster {target} Totals Over Time"

data = [go.Scatter(
    x=df_cross.index, 
    y=df_cross[col],
    name=col,
    line={"width": 1.5},
) for col in df_cross.columns]

layout = go.Layout(
    title=title,
    colorway=px.colors.qualitative.Antique,
    height=600,
    width=800,
    yaxis={"title": "Monster Count"},
    xaxis={"title": "Time Stamp"},
)

figure = go.Figure(data, layout)
figure.show()

## Stacked Bar Chart Crosstab: Rarity by Level

Dynaically add the name of the target to the title of the y-axis.

In [40]:
feature = "Level"  # ["Level", "Type", "Rarity"]
target = "Rarity"  # ["Level", "Type", "Rarity"]

df = db.get_df()

df_cross = pd.crosstab(df[feature], df[target])

title = f"{target} by {feature}"

data = [
    go.Bar(name=col, x=df_cross.index, y=df_cross[col])
    for col in df_cross.columns
]

layout = go.Layout(
    title=title,
    colorway=px.colors.qualitative.Antique,
    height=600,
    width=810,
    barmode="stack",
    yaxis={"title": "Monster Count"},
    xaxis={'title': feature}
)

figure = go.Figure(data, layout)

figure.show()

## Altair

In [41]:
import altair as alt

In [42]:
x_axis = "Health"  # ["Energy", "Sanity", "Health"]
y_axis = "Energy"  # ["Energy", "Sanity", "Health"]
target = "Rarity"  # ["Rarity", "Level", "Type"]
rarity = "All"     # ["All", "Rank 0", ... "Rank 5"]

monsters = db.get_df().drop(columns=['_id'])

if rarity != "All":
    monsters = monsters[monsters['Rarity'] == rarity]

graph = alt.Chart(
    monsters,
    title=f"{rarity} Monsters",
).mark_circle(size=100).encode(
    x=alt.X(
        x_axis,
        axis=alt.Axis(title=x_axis),
    ),
    y=alt.Y(
        y_axis,
        axis=alt.Axis(title=y_axis),
    ),
    color=target,
    tooltip=alt.Tooltip(list(monsters.columns)),
).properties(
    height=500,
    width=500,
)

graph

alt.Chart(...)

### Abstraction, Encapsulation, Polymophism

Below is one example of an abstraction that encasulates a graph and extends some customization points. Here we'll use a functional interface, but classes work too. 

You can parameterize every aspect of the graph by adding function arguments. Be mindfull, you don't want to over-do it here. Keep your calling signature simple and usable. Provide good defaults and well named arguments, and your users will enjoy using your code. Make it super complicated and they may as well just use Altair themselves.

A good interface should always encapsulate the core logic in such a way that the rest of the app is totally unaware of how it works, but can still interact with the core logic in a general way. One might say that the interface is more abstract than the core logic it encapsulates. At this higher abstraction level it becomes easier to replace our core logic without disrupting parallel development on other parts of the app. And now a word from our sponsor, Polymorphism.

One hypothetical example of Polymorphism is if we designed more than one graph, possibly with two different graphing libraries. Then gave them compatible interfaces. This gives us the ability to trade one graph library for another without rewriting the whole app. We could do that at any time during development without disrupting anything.

In [43]:
def scatter(x_axis="Health", y_axis="Energy", target="Rarity", rarity="All"):

    monsters = db.get_df().drop(columns=['_id'])

    if rarity != "All":
        monsters = monsters[monsters['Rarity'] == rarity]

    graph = alt.Chart(
        monsters,
        title=f"{rarity} Monsters",
    ).mark_circle(size=200).encode(
        x=alt.X(
            x_axis,
            axis=alt.Axis(title=x_axis),
        ),
        y=alt.Y(
            y_axis,
            axis=alt.Axis(title=y_axis),
        ),
        color=target,
        tooltip=alt.Tooltip(list(monsters.columns)),
    ).properties(
        height=500,
        width=500,
    )

    return graph

Try other columns below. In terms of data science, what's the most interesting graph you can make with the scatter_plot function? 

What feature(s) are missing from this function? 

What's cool about designing software this way? 

What's lacking about designing software this way?

In [60]:
scatter_plot = scatter(
    x_axis="Health",
    y_axis="Energy",
    target="Type", 
    rarity="Rank 0",
)
scatter_plot

alt.Chart(...)

Same graph as above as a JSON file...

Juggeling json is tricky until you get the hang of it, then it's really easy!

In [54]:
# The Altair Library provides the `.to_json()` method.
# This creates a dirty json string. In the next section we'll see about fixing it.
scatter_plot_json = scatter_plot.to_json()
scatter_plot_json

'{\n  "$schema": "https://vega.github.io/schema/vega-lite/v4.8.1.json",\n  "config": {\n    "view": {\n      "continuousHeight": 300,\n      "continuousWidth": 400\n    }\n  },\n  "data": {\n    "name": "data-bc909eade4e963ccacb1652308ad0036"\n  },\n  "datasets": {\n    "data-bc909eade4e963ccacb1652308ad0036": [\n      {\n        "Damage": "4d2+3",\n        "Energy": 8.1,\n        "Health": 7.73,\n        "Level": 4,\n        "Name": "Dust Spirit",\n        "Rarity": "Rank 0",\n        "Sanity": 8.62,\n        "Time Stamp": "2022-03-31 07:23:55",\n        "Type": "Fey"\n      },\n      {\n        "Damage": "12d2",\n        "Energy": 24.32,\n        "Health": 24.01,\n        "Level": 12,\n        "Name": "Emerald Dragon",\n        "Rarity": "Rank 0",\n        "Sanity": 23.85,\n        "Time Stamp": "2022-03-31 07:23:55",\n        "Type": "Dragon"\n      },\n      {\n        "Damage": "13d2+1",\n        "Energy": 25.26,\n        "Health": 26.77,\n        "Level": 13,\n        "Name": "Dj

# Data I/O Juggling

### JSON Library

In [55]:
import json

The JSON python library has 4 functions to help with JSON I/O
- 2 for File I/O
    - `json.load` -> load a dict from JSON data in a file
    - `json.dump` -> takes a dict and saves JSON data to a file. Best use context manager.
- 2 for Memory I/O
    - `json.loads` -> creates a dict from JSON data in memory (some variable)
    - `json.dumps`  -> turns a dict into JSON data in memory (some variable)


In [56]:
# json.loads will turn dirty json into a dictionary
scatter_plot_dict = json.loads(scatter_plot_json)
scatter_plot_dict

{'$schema': 'https://vega.github.io/schema/vega-lite/v4.8.1.json',
 'config': {'view': {'continuousHeight': 300, 'continuousWidth': 400}},
 'data': {'name': 'data-bc909eade4e963ccacb1652308ad0036'},
 'datasets': {'data-bc909eade4e963ccacb1652308ad0036': [{'Damage': '4d2+3',
    'Energy': 8.1,
    'Health': 7.73,
    'Level': 4,
    'Name': 'Dust Spirit',
    'Rarity': 'Rank 0',
    'Sanity': 8.62,
    'Time Stamp': '2022-03-31 07:23:55',
    'Type': 'Fey'},
   {'Damage': '12d2',
    'Energy': 24.32,
    'Health': 24.01,
    'Level': 12,
    'Name': 'Emerald Dragon',
    'Rarity': 'Rank 0',
    'Sanity': 23.85,
    'Time Stamp': '2022-03-31 07:23:55',
    'Type': 'Dragon'},
   {'Damage': '13d2+1',
    'Energy': 25.26,
    'Health': 26.77,
    'Level': 13,
    'Name': 'Djinni',
    'Rarity': 'Rank 0',
    'Sanity': 25.07,
    'Time Stamp': '2022-03-31 07:23:55',
    'Type': 'Elemental'},
   {'Damage': '8d2',
    'Energy': 16.85,
    'Health': 15.57,
    'Level': 8,
    'Name': 'Steam Arc

In [57]:
# json dumps will create a clean json string from a dict
scatter_plot_json = json.dumps(scatter_plot_dict)
scatter_plot_json

'{"$schema": "https://vega.github.io/schema/vega-lite/v4.8.1.json", "config": {"view": {"continuousHeight": 300, "continuousWidth": 400}}, "data": {"name": "data-bc909eade4e963ccacb1652308ad0036"}, "datasets": {"data-bc909eade4e963ccacb1652308ad0036": [{"Damage": "4d2+3", "Energy": 8.1, "Health": 7.73, "Level": 4, "Name": "Dust Spirit", "Rarity": "Rank 0", "Sanity": 8.62, "Time Stamp": "2022-03-31 07:23:55", "Type": "Fey"}, {"Damage": "12d2", "Energy": 24.32, "Health": 24.01, "Level": 12, "Name": "Emerald Dragon", "Rarity": "Rank 0", "Sanity": 23.85, "Time Stamp": "2022-03-31 07:23:55", "Type": "Dragon"}, {"Damage": "13d2+1", "Energy": 25.26, "Health": 26.77, "Level": 13, "Name": "Djinni", "Rarity": "Rank 0", "Sanity": 25.07, "Time Stamp": "2022-03-31 07:23:55", "Type": "Elemental"}, {"Damage": "8d2", "Energy": 16.85, "Health": 15.57, "Level": 8, "Name": "Steam Archfey", "Rarity": "Rank 0", "Sanity": 15.07, "Time Stamp": "2022-03-31 07:23:55", "Type": "Fey"}, {"Damage": "12d2", "Energy

Context managers help manage resources like files that need to be closed when we're done with them. The context manager automatically closes its resource when it goes out of scope. Two context managers are defined below, they begin with the `with` keyword.

In [58]:
# json dump will save a json file from a dict
with open("scatter_plot.json", "w") as file:
    json.dump(scatter_plot_dict, file)

In [59]:
# json load will open a json file and turn it into a dict
with open("scatter_plot.json", "r") as file:
    json_dict = json.load(file)
json_dict

{'$schema': 'https://vega.github.io/schema/vega-lite/v4.8.1.json',
 'config': {'view': {'continuousHeight': 300, 'continuousWidth': 400}},
 'data': {'name': 'data-bc909eade4e963ccacb1652308ad0036'},
 'datasets': {'data-bc909eade4e963ccacb1652308ad0036': [{'Damage': '4d2+3',
    'Energy': 8.1,
    'Health': 7.73,
    'Level': 4,
    'Name': 'Dust Spirit',
    'Rarity': 'Rank 0',
    'Sanity': 8.62,
    'Time Stamp': '2022-03-31 07:23:55',
    'Type': 'Fey'},
   {'Damage': '12d2',
    'Energy': 24.32,
    'Health': 24.01,
    'Level': 12,
    'Name': 'Emerald Dragon',
    'Rarity': 'Rank 0',
    'Sanity': 23.85,
    'Time Stamp': '2022-03-31 07:23:55',
    'Type': 'Dragon'},
   {'Damage': '13d2+1',
    'Energy': 25.26,
    'Health': 26.77,
    'Level': 13,
    'Name': 'Djinni',
    'Rarity': 'Rank 0',
    'Sanity': 25.07,
    'Time Stamp': '2022-03-31 07:23:55',
    'Type': 'Elemental'},
   {'Damage': '8d2',
    'Energy': 16.85,
    'Health': 15.57,
    'Level': 8,
    'Name': 'Steam Arc